In [ ]:
import time
from bs4 import BeautifulSoup
import regex as re
import requests
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv

In [ ]:
def cargarElementosTratados(rutaDataSet):
    try:
        df = pd.read_csv(rutaDataSet, header = None, names = ['id', 'appId'])
        data_list = list(df['id'])
    except Exception as e:
        data_list = []

    return data_list 

In [ ]:
def crearListaElementosATratar(tratados, inicio = 1, limite = 100):
    lista = list(range(inicio, inicio + limite))
    
    return [x for x in lista if x not in tratados]

In [ ]:
def rastrearHtml(html):
    appId = 'na'
    soup = BeautifulSoup(html, features='lxml')
  
    #Id de la aplicación
    try:
        tag = soup.find('a', {'class': 'link main-link'})
        appId = tag['href'][tag['href'].find('=') + 1:] 
    except Exception as e:
        appId = 'na'

    return appId

In [ ]:
HEADER = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, sdch, br",
    "Accept-Language": "en-US,en;q=0.8",
    "Cache-Control": "no-cache",
    "dnt": "1",
    "Pragma": "no-cache",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
}
URL_BASE = 'https://reports.exodus-privacy.eu.org/es/reports/'
fichero = '/content/drive/MyDrive/exodus/exodusAppId.csv'
INICIO = 1
LIMITE = 153373

#Inicializar lista de los elementos ya tratados a partir del json del dataset.
tratados_list = cargarElementosTratados(fichero)
#Inicializar índice de elementos
elem = 0
#Crear lista de elementos a tratar(Serie_elementos_tratados, inicio, limite)
lista = crearListaElementosATratar(tratados_list, INICIO, LIMITE)
#Crear el diccionario donde se recogeran los elementos procesados de la sesión.
scrap_json = {}

while elem < len(lista): #Mientras existan elementos en la lista
    web = None
    url = URL_BASE + str(lista[elem]) + '/'

    try: #Manejo de posibles excepciones causadas por la petición de la página gestionando tiempos, repeticiones o paradas
        web = requests.get(url, headers = HEADER)
    except requests.exceptions.ConnectionError as e:
        print('Error de conexión procesando ' + url + '. ' + str(e))
    except requests.exceptions.ConnectTimeout as e:
        print('Error de conexión procesando ' + url + '. ' + str(e))
    except requests.exceptions.ProxyError as e:
        print('Error fatal de proxy procesando ' + url + '. ' + str(e))
    except requests.exceptions.SSLError as e:
        print('Error fatal de SSL procesando ' + url + '. ' + str(e))
    except Exception as e:
        print('Error fatal no controlado procesando ' + url + '. ' + str(e))

    if web != None: #Si se ha recuperado información del servidor, comprobamos su status.
        if (web.status_code >= 500) and (web.status_code < 600): #Errores en el servidor
            print('Error ' + str(web.status_code) + ' de servidor producido en la request procesando ' + url)
            elem += 1
        if (web.status_code >= 400) and (web.status_code < 500): #Errores en el cliente > Avisar y volvemos a intentarlo tras un tiempo hasta agotar intentos y pasar al siguiente
            print('Error ' + str(web.status_code) + ' de cliente producido en la request procesando ' + url)
            elem += 1
        if (web.status_code >= 300) and (web.status_code < 400): #Errores de redirección > Paramos el rastreo de esta url y seguimos.
            print('Error ' + str(web.status_code) + ' de redirección en la request procesando ' + url)
            elem += 1
        if (web.status_code > 200) and (web.status_code < 300): #Petición correcta con incidencias > Avisar y volvemos a intentarlo tras un tiempo hasta agotar intentos y pasar al siguiente.
            print('Incidencia ' + str(web.status_code) + ' tras petición correcta procesando  ' + url)
            elem += 1
        if web.status_code == 200: #Petición correcta y html a nuestra disposición
            app = rastrearHtml(web.content)
            if app != 'na': #Si se ha procesado la web correctamente y extraído la información
                fila = [str(lista[elem]), app]
                with open(fichero, 'a', newline = '') as f:
                    writer = csv.writer(f)
                    writer.writerow(fila)
                f.close()
                elem += 1
                #print('Rastreada url ' + url + ' con éxito')
                time.sleep(0.5)
            else: #Si no se ha procesado por un fallo del parseador del html se apunta la incidencia y se procede con el siguiente
                elem += 1
    else: #Si no se ha recuperado información del servidor
        elem += 1

print('Rastreo finalizado')

#Actualizar el diccionario de los elmentos ya tratados con los rastreados en esta sesión y guardar el dataset y las incidencias
#tratados_dict.update(scrap_json)
#
#try:
#    with open(fichero, 'w') as outfile: #Al acabar, volcar el nuevo fichero con el dataset.
#        json.dump(tratados_dict, outfile)
#except Exception as e:
#    print('Error escribiendo dataset')
#    print(e)